In [8]:
import pandas as pd

# 读取Excel文件
df = pd.read_excel('Questions with labels.xlsx')

# 创建一个空的DataFrame来保存结果
result_df = pd.DataFrame(columns=['Question','Question_type', 'Entity1:Object'])

# 遍历原始DataFrame的每一行
for index, row in df.iterrows():
    if row[2] == 'Spectral Detection Method':
        # 如果第三列的值为"Spectral Detection Method"，则提取第二列和第四列的值
        new_row = pd.DataFrame({'Question': [row[1]], 'Question_type':[row[2]],'Entity1:Object': [row[3]]})
        result_df = pd.concat([result_df, new_row], ignore_index=True)

# 保存结果到新的Excel文件
result_df.to_excel('Answers_type1.xlsx', index=False)

C:\Users\Raytine\AppData\Local\Temp\ipykernel_19228\2371528191.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[2] == 'Spectral Detection Method':
C:\Users\Raytine\AppData\Local\Temp\ipykernel_19228\2371528191.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row = pd.DataFrame({'Question': [row[1]], 'Question_type':[row[2]],'Entity1:Object': [row[3]]})


Dense Retrieval

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 读取Excel文件
df1 = pd.read_excel('Answers_type1.xlsx')
df2 = pd.read_excel("Label_ver7_updated.xlsx")

# 定义TF-IDF向量化器
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)

# 准备语料库
corpus = [str(row[5]) + " " + str(row[6]) + " " + str(row[14]).lower() for _, row in df2.iterrows()]

# 计算TF-IDF矩阵
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# 定义余弦相似度检索函数
def cosine_retrieval(keywords, dataframe, top_n=5):
    # 将关键词转换为小写
    keywords = keywords.strip().lower()
    
    # 构建查询文本的TF-IDF向量
    query_vector = tfidf_vectorizer.transform([keywords])
    
    # 计算查询文本与语料库中每个文档的余弦相似度
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # 获取相似度最高的top_n行的索引
    top_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # 返回相关度最高的论文标题以及同一行第12列的值
    return dataframe.iloc[top_indices][[dataframe.columns[1], dataframe.columns[11]]]

# 遍历df1中每行的第三列值作为关键词进行检索
results = []
for index, row in df1.iterrows():
    keywords = row[2]  # 第三列的索引为2
    relevant_rows = cosine_retrieval(keywords, df2)
    results.append(relevant_rows.values.flatten())  # 扁平化数组以便合并到DataFrame中

# 将结果转换为DataFrame并与df1合并
results_df = pd.DataFrame(results, columns=['Top1', 'Top1_Info', 'Top2', 'Top2_Info', 'Top3', 'Top3_Info', 'Top4', 'Top4_Info', 'Top5', 'Top5_Info'])
df1 = pd.concat([df1, results_df], axis=1)

# 保存更新后的DataFrame到新的Excel文件
df1.to_excel('Answers_type1.xlsx', index=False)

print("处理完成，更新后的文件已保存为 'Answers_type1.xlsx'")



C:\Users\Raytine\AppData\Local\Temp\ipykernel_19228\1431122909.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  keywords = row[2]  # 第三列的索引为2
C:\Users\Raytine\AppData\Local\Temp\ipykernel_19228\1431122909.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  corpus = [str(row[5]) + " " + str(row[6]) + " " + str(row[14]).lower() for _, row in dataframe.iterrows()]
C:\Users\Raytine\AppData\Local\Temp\ipykernel_19228\1431122909.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

处理完成，更新后的文件已保存为 'updated_Answers_type1.xlsx'


转化光谱序号为文字

In [1]:
import pandas as pd

# 定义映射关系
conversion_dict = {
    '1': ['ultravio spectroscopy(uv)'],
    '2': ['Visible light spectroscopy(vis)'],
    '3.1': ['near-infrared spectroscopy(nir)'],
    '3.2': ['mid-infrared spectroscopy(mir)'],
    '3.3': ['Fourier transform infrared spectroscopy(ftir)'],
    '3.4': [ 'long-wave infrared spectroscopy(lir)'],
    '4': ['Raman spectroscopy'],
    '5': ['Laser-induced breakdown spectroscopy(LIBS)'],
    '6': ['terahertz spectroscopy(THz)'],
    '7': ['fluorescence spectroscopy'],
    '8': ['hyperspectral imaging technology'],
    '9': ['else']
}

# 读取 Excel 文件
file_path = 'Answers_type1.xlsx'  # 更改为你的文件路径
df = pd.read_excel(file_path)

# 转换数字到相应的文字列表
def convert_numbers_to_text(num):
    num = str(num)
    # 处理可能包含多个数字的情况，例如 "3.1, 3.2"
    parts = num.split(',')
    texts = []
    for part in parts:
        part = part.strip()  # 清除空白
        if part in conversion_dict:
            texts.extend(conversion_dict[part])
        else:
            texts.append('Unknown')
    return ', '.join(texts)

# 指定要转换的列索引
columns_to_convert = [5]  # Python索引为0开始，所以第5列是索引4，依此类推

# 应用转换函数到指定列
for col_index in columns_to_convert:
    df.iloc[:, col_index] = df.iloc[:, col_index].apply(convert_numbers_to_text)

# 保存修改后的 DataFrame 到新的 Excel 文件
df.to_excel('Answers_type1.xlsx', index=False)


特定模板生成答案并转换成json

In [1]:
import pandas as pd
import json

# 加载Excel文件
file_path = 'Answers_type1.xlsx'  # 请根据实际情况修改文件路径
sheet_name = 'Sheet1'  # 根据实际情况修改工作表名称
df = pd.read_excel(file_path, sheet_name=sheet_name)

# 生成答案
def generate_answer(question, row):
    titles = [row[title] for title in ['Paper1', 'Paper2', 'Paper3', 'Paper4', 'Paper5']]
    spectra = [row[spectrum] for spectrum in ['Spectrum of paper1', 'Spectrum of paper2', 'Spectrum of paper3', 'Spectrum of paper4', 'Spectrum of paper5']]
    
    answer = f"The following are five papers related to the question (in descending order of relevance):\n"
    for i, (title, spectrum) in enumerate(zip(titles, spectra), start=1):
        answer += f"{i}.{title}.\nThe spectroscopy used in the paper: {spectrum}\n"
    
    return answer

# 准备JSON数据
json_data = []
for index, row in df.iterrows():
    question = row['Question']
    question_type = row['Question_type']  # 假设第二列是问题类型
    entity = row['Entity1:Object']  # 假设第三列是实体对象
    answer = generate_answer(question, row)
    
    # 收集相关论文和光谱方法
    knowledge_list = []
    for i in range(1, 6):
        paper_info = f"Paper{i}: {row[f'Paper{i}']}, Spectroscopy: {row[f'Spectrum of paper{i}']}"
        knowledge_list.append(paper_info)

# 将列表中的所有元素合并为一个字符串，每个元素之间用 "###" 分隔
    Knowledge = "\n".join(knowledge_list)

    # 构造单个JSON对象
    json_object = {
        "Instruction":'Based on the information of Knowledge in this section, answer the following question concisely and professionally. The format of the answer should be the same as in the output, i.e., five different papers related to the question and the spectroscopic method used should be answered. The content of the answer must be derived from the knowledge, and the title of the paper answered must be exactly the same as the knowledge, and cannot be generated arbitrarily. Qu',
        "Question": question,
        "Question_type": question_type,
        "Entity1:Object": entity,
        "Entity2:Spectrum":"not found",
        "Knowledge":Knowledge,
        "Output": answer
    }
    json_data.append(json_object)

# 将数据输出为JSON格式
json_output = json.dumps(json_data, indent=4)
print(json_output)

# 如果需要，可以将JSON数据保存到文件中
with open('Answers_type1.json', 'w') as json_file:
    json_file.write(json_output)


[
    {
        "Instruction": "Based on the information of Knowledge in this section, answer the question concisely and professionally. The format of the answer should be the same as in the output, i.e., five different papers related to the question and the spectroscopic method used should be answered. The content of the answer must be derived from the knowledge, and the title of the paper answered must be exactly the same as the knowledge, and cannot be generated arbitrarily.",
        "Question": "What kind of spectral detection method was used in the related research of predicting Chinese black tea ranking?",
        "Question_type": "Spectral Detection Method",
        "Entity1:Object": "Chinese black tea",
        "Entity2:Spectrum": "not found",
        "Knowledge": "Paper1: Cognitive spectroscopy for evaluating Chinese black tea grades (Camellia sinensis): near-infrared spectroscopy and evolutionary algorithms, Spectroscopy: near-infrared spectroscopy(nir)\nPaper2: Rapid detect